In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
RND_ST = 16383
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.duplicated().sum()

0

In [4]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [5]:
df.is_ultra.mean()

0.30647168637212197

### Вывод:

Данные готовы для дальнейшей работы.  
Столбец 'is_ultra' содержит несбалансированные данные: около 30% имеют значение 1, остальные - 0
----

## Разбейте данные на выборки

In [6]:
features = df.drop('is_ultra', axis=1)
target = df.is_ultra

train_features, other_features, train_target, other_target = \
    train_test_split(features, target, 
                     test_size=0.40, 
                     random_state=RND_ST, 
                     stratify=target)

valid_features, test_features, valid_target, test_target = \
    train_test_split(other_features, other_target, 
                     test_size=0.50, 
                     random_state=RND_ST, 
                     stratify=other_target)


In [7]:
print('train_target:',train_target.count(), train_target.mean())
print('valid_target:',valid_target.count(), valid_target.mean())
print(' test_target:',test_target.count(), test_target.mean())

train_target: 1928 0.3065352697095436
valid_target: 643 0.30637636080870917
 test_target: 643 0.30637636080870917


### Вывод:

Исходные данные разбиты на 3 части:  
- 60% выборка для обучения моделей
- 20% выборка для валидации
- 20% выборка для окончательного тестирования моделей  

Разделение выполнено так, чтобы баланс целевого параметра ('is_ultra') сохранился во всех выборках  

---


## Исследуйте модели

### Дерево решений:

In [8]:
best_model = None
best_result = 0
worst_result = 1
for depth in range(2, 9):
    for prm in range(1,5):
        model = DecisionTreeClassifier(random_state=RND_ST,
                                       class_weight='balanced',
                                       min_samples_leaf=prm,
                                       max_depth=depth)
        model.fit(train_features, train_target) 
        result = model.score(valid_features, valid_target)
        if result > best_result:
            best_model = model 
            best_result = result 
        if result < worst_result: worst_result=result

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Accuracy наихудшей модели на валидационной выборке:", worst_result)
print("Лучшая модель:")
print(best_model)
model_DT = best_model

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625
Accuracy наихудшей модели на валидационной выборке: 0.7371695178849145
Лучшая модель:
DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=16383, splitter='best')


### Случайный лес:

In [9]:
best_model = None
best_result = 0
worst_result = 1
for est in range(1, 15):
    for depth in range(2, 11):
        for prm in range(2,5):
            model = RandomForestClassifier(random_state=RND_ST,
                                           class_weight='balanced',
                                           max_depth=depth,
                                           min_samples_split=prm,
                                           n_estimators=est)     
            model.fit(train_features, train_target) 
            result = model.score(valid_features, valid_target) 
            if result > best_result:
                best_model = model 
                best_result = result 
            if result < worst_result: worst_result=result

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Accuracy наихудшей модели на валидационной выборке:", worst_result)
print("Лучшая модель:")
print(best_model)
model_RF = best_model

Accuracy наилучшей модели на валидационной выборке: 0.8227060653188181
Accuracy наихудшей модели на валидационной выборке: 0.7091757387247278
Лучшая модель:
RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=3, min_weight_fraction_leaf=0.0,
                       n_estimators=8, n_jobs=None, oob_score=False,
                       random_state=16383, verbose=0, warm_start=False)


### Логистическая регрессия:

In [10]:
best_model = None
best_result = 0
worst_result = 1
for prm_C in range(10, 150, 5):
    for prm_is in range(90, 110):
        model = LogisticRegression(random_state=RND_ST,
                                   #class_weight='balanced',
                                   solver='liblinear', 
                                   intercept_scaling=prm_is/100,
                                   fit_intercept=True,
                                   C=prm_C/100,
                                   penalty='l2')     
        model.fit(train_features, train_target) 
        result = model.score(valid_features, valid_target) 
        if result > best_result:
            best_model = model 
            best_result = result
        if result < worst_result: worst_result=result

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Accuracy наихудшей модели на валидационной выборке:", worst_result)
print("Лучшая модель:")
print(best_model)
model_LR = best_model

Accuracy наилучшей модели на валидационной выборке: 0.7371695178849145
Accuracy наихудшей модели на валидационной выборке: 0.6951788491446346
Лучшая модель:
LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1.06, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=16383, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)


### Вывод:

Рассмотрены 3 модели:
- Дерево решений
- Случайный лес
- Логистическая регрессия  

Выполнен подбор гиперпараметров для всех моделей так,  
чтобы на валидационной выборке получить максимальную точность.

---

## Проверьте модель на тестовой выборке

In [11]:
# дерево решений:
print(model_DT)
model_DT.score(test_features, test_target)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=16383, splitter='best')


0.7900466562986003

In [12]:
# случайный лес:
print(model_RF)
model_RF.score(test_features, test_target)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=3, min_weight_fraction_leaf=0.0,
                       n_estimators=8, n_jobs=None, oob_score=False,
                       random_state=16383, verbose=0, warm_start=False)


0.7869362363919129

In [13]:
# логистическая регрессия:
print(model_LR)
model_LR.score(test_features, test_target)

LogisticRegression(C=1.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1.06, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=16383, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)


0.7402799377916018

### Вывод:

На тестовой выборке две модели (дерево решений и случайный лес)  
показали точность предсказаний около 79%.  
Модель логистическая регрессия показала только 74% точность.   

---


## (бонус) Проверьте модели на адекватность

Целевой параметр не сбалансирован и содержит около 70% нулевых значений,  
и, например, если модель всегда будет предсказывать значение 0,  
то она будет иметь точность около 70%.  

Рассмотрим другие метрики:

In [14]:
def calc_metrics(model):
    trg = test_target; prd = model.predict(test_features)
    print('\nconfusion_matrix:\n\n', confusion_matrix(trg, prd))
    print('\nclassification_report:\n\n', classification_report(trg, prd))
    print('\nbalanced_accuracy:', round(balanced_accuracy_score(trg, prd),2))
    print('-------------------------------------------------------\n')

In [15]:
print('Дерево решений:'); calc_metrics(model_DT)
print('Случайный лес:'); calc_metrics(model_RF)
print('Логистическая регрессия:'); calc_metrics(model_LR)

Дерево решений:

confusion_matrix:

 [[412  34]
 [101  96]]

classification_report:

               precision    recall  f1-score   support

           0       0.80      0.92      0.86       446
           1       0.74      0.49      0.59       197

    accuracy                           0.79       643
   macro avg       0.77      0.71      0.72       643
weighted avg       0.78      0.79      0.78       643


balanced_accuracy: 0.71
-------------------------------------------------------

Случайный лес:

confusion_matrix:

 [[388  58]
 [ 79 118]]

classification_report:

               precision    recall  f1-score   support

           0       0.83      0.87      0.85       446
           1       0.67      0.60      0.63       197

    accuracy                           0.79       643
   macro avg       0.75      0.73      0.74       643
weighted avg       0.78      0.79      0.78       643


balanced_accuracy: 0.73
-------------------------------------------------------

Логистическ

### Вывод:

Все три модели неплохо классифицируют значение 0,  
но значение 1 удовлетворительно предсказывают только случайный лес и дерево решений.  
Кроме того, логистическая регрессия не достигает требуемой (0.75) точности.

---

## Общий вывод:

__Наиболее подходящей моделью для прогнозироввания тарифа является случайный лес с параметрами:__ 
```
RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=3, min_weight_fraction_leaf=0.0,
                       n_estimators=8, n_jobs=None, oob_score=False,
                       random_state=16383, verbose=0, warm_start=False)
```

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
